In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# %pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


/usr/local/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# df = pd.read_csv('D:\IET software-CC-CL\Commit Classification\experiment\Commit_dataset.csv', encoding="cp1252")

# label2id = {'a':'Adaptive','p':'Perfective','c':'Corrective'}
# df = df.replace({"3_labels": label2id})
# df = pd.read_csv(r'dataset.csv',engine="python")
# df['text'] = "['DIFF]" + df['diff']
# # df = df.replace({"2_labels": label2id})
# print(df)

train = pd.read_csv('train.csv',index_col=0)
train = train.rename(columns={'3_labels':'label','comment':'text'})
train.fillna(0, inplace=True)
test = pd.read_csv('test.csv',index_col=0)
test = test.rename(columns={'3_labels':'label','comment':'text'})
test.fillna(0, inplace=True)
print(len(test))



538


In [3]:
# train_change_feautre = train.iloc[:, 4:]
# test_change_feautre = test.iloc[:, 4:]

# train_code_change = train_change_feautre.values
# test_code_change = test_change_feautre.values

In [4]:
# !pip install setfit
from commitfit import get_templated_dataset,sample_dataset
from datasets import Dataset, load_metric

In [5]:
Dataset_train = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

In [6]:
train_dataset = get_templated_dataset(Dataset_train, candidate_labels=['Corrective','Adaptive','Perfective'], sample_size=8)

In [7]:
# list(train['message'].astype(str).values)
train_dataset

Dataset({
    features: ['commit_id', 'project', 'text', 'label', 'ADDING_ATTRIBUTE_MODIFIABILITY', 'ADDING_METHOD_OVERRIDABILITY', 'ADDITIONAL_CLASS', 'ADDITIONAL_FUNCTIONALITY', 'ADDITIONAL_OBJECT_STATE', 'ALTERNATIVE_PART_DELETE', 'ALTERNATIVE_PART_INSERT', 'ATTRIBUTE_RENAMING', 'ATTRIBUTE_TYPE_CHANGE', 'CLASS_RENAMING', 'COMMENT_DELETE', 'COMMENT_INSERT', 'COMMENT_MOVE', 'COMMENT_UPDATE', 'CONDITION_EXPRESSION_CHANGE', 'DECREASING_ACCESSIBILITY_CHANGE', 'DOC_DELETE', 'DOC_INSERT', 'DOC_UPDATE', 'INCREASING_ACCESSIBILITY_CHANGE', 'METHOD_RENAMING', 'PARAMETER_DELETE', 'PARAMETER_INSERT', 'PARAMETER_ORDERING_CHANGE', 'PARAMETER_RENAMING', 'PARAMETER_TYPE_CHANGE', 'PARENT_CLASS_CHANGE', 'PARENT_CLASS_DELETE', 'PARENT_CLASS_INSERT', 'PARENT_INTERFACE_CHANGE', 'PARENT_INTERFACE_DELETE', 'PARENT_INTERFACE_INSERT', 'REMOVED_CLASS', 'REMOVED_FUNCTIONALITY', 'REMOVED_OBJECT_STATE', 'REMOVING_ATTRIBUTE_MODIFIABILITY', 'REMOVING_CLASS_DERIVABILITY', 'REMOVING_METHOD_OVERRIDABILITY', 'RETURN_T

In [8]:
len(train)

1255

In [9]:
train['label'].value_counts()

label
Corrective    422
Perfective    420
Adaptive      413
Name: count, dtype: int64

In [10]:
test['label'].value_counts()

label
Corrective    181
Perfective    180
Adaptive      177
Name: count, dtype: int64

In [11]:
len(train_dataset)

1279

In [12]:
from sklearn import  metrics

def compute_metrics(y_pred, y_test):
    # print(y_pred,y_test)
    # classification_report = metrics.classification_report(y_test,y_pred)
    accuracy_score = metrics.accuracy_score(y_test,y_pred)
    precision_score = metrics.precision_score(y_test,y_pred, average='weighted')
    recall_score = metrics.recall_score(y_test,y_pred,average='weighted')
    f1_score = metrics.f1_score(y_test,y_pred,average='weighted')

    # return {"classification_report": classification_report}
    return {"accuracy": accuracy_score, "precision": precision_score,"recall": recall_score, "f1":f1_score}

In [13]:
train_dataset

Dataset({
    features: ['commit_id', 'project', 'text', 'label', 'ADDING_ATTRIBUTE_MODIFIABILITY', 'ADDING_METHOD_OVERRIDABILITY', 'ADDITIONAL_CLASS', 'ADDITIONAL_FUNCTIONALITY', 'ADDITIONAL_OBJECT_STATE', 'ALTERNATIVE_PART_DELETE', 'ALTERNATIVE_PART_INSERT', 'ATTRIBUTE_RENAMING', 'ATTRIBUTE_TYPE_CHANGE', 'CLASS_RENAMING', 'COMMENT_DELETE', 'COMMENT_INSERT', 'COMMENT_MOVE', 'COMMENT_UPDATE', 'CONDITION_EXPRESSION_CHANGE', 'DECREASING_ACCESSIBILITY_CHANGE', 'DOC_DELETE', 'DOC_INSERT', 'DOC_UPDATE', 'INCREASING_ACCESSIBILITY_CHANGE', 'METHOD_RENAMING', 'PARAMETER_DELETE', 'PARAMETER_INSERT', 'PARAMETER_ORDERING_CHANGE', 'PARAMETER_RENAMING', 'PARAMETER_TYPE_CHANGE', 'PARENT_CLASS_CHANGE', 'PARENT_CLASS_DELETE', 'PARENT_CLASS_INSERT', 'PARENT_INTERFACE_CHANGE', 'PARENT_INTERFACE_DELETE', 'PARENT_INTERFACE_INSERT', 'REMOVED_CLASS', 'REMOVED_FUNCTIONALITY', 'REMOVED_OBJECT_STATE', 'REMOVING_ATTRIBUTE_MODIFIABILITY', 'REMOVING_CLASS_DERIVABILITY', 'REMOVING_METHOD_OVERRIDABILITY', 'RETURN_T

In [14]:
train

,commit_id,project,text,label,ADDING_ATTRIBUTE_MODIFIABILITY,ADDING_METHOD_OVERRIDABILITY,ADDITIONAL_CLASS,ADDITIONAL_FUNCTIONALITY,ADDITIONAL_OBJECT_STATE,ALTERNATIVE_PART_DELETE,...,Extract Variable,Inline Variable,Parametrize Variable,Rename Variable,Replace Variable with Attribute,Change Variable Type,Move and Rename Class,Move Class,Extract and Move Method,Move Source Folder
1,0950c46beda335819928585f1262dfe1dca78a0b,ReactiveX-RxJava,Trying to extend the Scheduler interface accor...,Adaptive,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
498,869a002cae63a4e8ab52ec7f2d15d5a2cfbe0c02,drools,[DROOLS-839] fix LogicTransformer with Accumul...,Corrective,0,0,1,9,1,1,...,0,0,0,1,0,1,0,0,0,0
1010,3ca94cabb68789cb20e37e0c9ecea7acc8b9487e,Valadoc,"libvaladoc/html: Add ""All known members inheri...",Adaptive,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1583,171accb43d709bd3960187686afbdf68f5239644,aeshell$aesh,[AESH-316] - print warning instead of exceptio...,Adaptive,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
1369,5b91442a260b42e3d47ef91a53015f2d0958a683,Delta Spike,remove unused private field\n,Perfective,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1411,f2757568dac2a6473f054101f80ac2b8f291ce4e,Vala,glib-2.0: add GLib.HashTable.foreach\n\nFixes ...,Adaptive,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1574,16cb7d8d59add7b3ef513bba0de6a4fc07e3bc52,aeshell$aesh,"search is more complete, support forward/rever...",Adaptive,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
597,14c657d448b6dd743806c4df2a321d58f4e0618e,kotlin,"Extract Function: Consider reference ""broken"" ...",Corrective,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1712,02668af08bebc5266b321ab1df2f45dfe656c813,duracloud$duracloud,First part of https://jira.duraspace.org/brows...,Perfective,0,0,0,1,0,0,...,0,0,0,1,0,0,0,1,0,0


In [15]:
model_id = "all-roberta-large-v1"

In [16]:
from commitfit import CommitFitModel
model = CommitFitModel.from_pretrained(model_id)

model_head.pkl not found in /all-roberta-large-v1, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [17]:
from commitfit import CommitFitTrainer

trainer = CommitFitTrainer(
    model=model,
    train_dataset=train_dataset,
    # train_code_change = train_change_feautre,
    # test_code_change = test_change_feautre,
    eval_dataset=test_dataset,
    metric = compute_metrics,
    num_iterations=20,
    num_epochs=1
)

In [18]:
%%time
trainer.train()
fewshot_metrics = trainer.evaluate()
fewshot_metrics

Generating Training Pairs: 100%|██████████| 20/20 [00:02<00:00,  7.64it/s]
***** Running training *****
  Num examples = 51160
  Num epochs = 1
  Total optimization steps = 3198
  Total train batch size = 16
Epoch: 100%|██████████| 1/1 [23:31<00:00, 1411.56s/it]
***** Running evaluation *****


CPU times: user 21min 56s, sys: 4min 20s, total: 26min 17s
Wall time: 23min 38s


{'accuracy': 0.8234200743494424,
 'precision': 0.8235163235253944,
 'recall': 0.8234200743494424,
 'f1': 0.8234118523135082}